Analysis of the winding parameters of a 1250 kVA Superconducting Transformer
-------------------------------------------------------------------------------------------------------------------

The transformer and its parameters originally published in DOI:10.1109/TASC.2014.2345349.

Main parameters of the  transformer:

- $S_n$ = 1250 kVA - nominal power
- $f$ = 50 Hz - network frequency
- $\epsilon$ = 5.8 % - short circuit impedance
- $U_1 / U_2$ = 10.5 kV / 0.4 kV
- $I_1 / I_2$ = 69 A / 1804 A
- Connection group: YNyn0

Independent design parameters for the two-winding transformer model:

- $r_c$ core radius : 155 mm
- $b_c$ flux density: 1.5 T
- $j_{in}$ current density in the inner winding: 4.05 $A/mm^2$ -- 100% filling factor assumed
- $j_{ou}$ current density in the outer winding: 6.2 $A/mm^2$
- $h_{in}$ the height of the inner winding: 342.5 mm
- $m_{gap}$ main gap - the distance between two main windings: 34.5 mm

Cooling System:

- Cryostat inner / outer diameter: 330/740 mm
- Operation temperature: 77 K


In [1]:
import os
import sys
parent_dir = os.path.split(os.getcwd())[0]
if parent_dir not in sys.path:
    sys.path.append(parent_dir)
    
from src.base_functions import turn_voltage

Calculating the turn voltage from the winding parameters:

In [19]:
# seconday - outer, primary inner winding
Np = 262.0 # turns in the primary winding
Up = 10500 # V 
utp = Up/Np/1.73
print("Turn voltage from primary winding data: ", round(utp,2), "V")

# secondary winding
Ns = 10
Us = 400
uts= Us/1.73/Ns
print("Turn voltage from secondary winding data:",round(uts,2), "V")

# BSSCO tape dimensions
w = 4.8  # mm
h = 0.32 # mm
A = 4.8*0.32 # 1.536 mm2

# Winding areas
a_lv = 13.5 * 342.5 # mm
a_hv = 8* 355

Nd = 22 # number of discs/pancakes
ff_lv = A*Np/a_lv*100
ff_hv = A*Ns*Nd/a_hv*100

print('Filling factor in LV (primary) winding', round(ff_lv,2), '%')
print('Filling factor in HV (secondary) winding',round(ff_hv,2), '%')

I1 = 69.
I2 = 1804.

ae_lv = (a_lv*(ff_lv/100))
ae_hv = (a_hv*(hh_hv/100))

j_lv = I2*Np/ae_lv
j_hv = I1*Nd*Ns/ae_hv

print(j_lv, ae_lv, a_lv)
print(j_hv, ae_hv, a_hv)


Turn voltage from primary winding data:  23.17 V
Turn voltage from secondary winding data: 23.12 V
Filling factor in LV (primary) winding 8.7 %
Filling factor in HV (secondary) winding 11.9 %
1174.4791666666665 402.432 4623.75
44.92187500000001 337.91999999999996


The core filling factor in the core's column was calculated from the other calculation of the turn voltage:

In [3]:
ut = turn_voltage(1.5, 155.0, 0.92, 50.0)
print('turn voltage:',round(ut,2), 'V')

turn voltage: 23.12 V


In [4]:
from importlib_resources import files
from src.two_winding_model import TransformerDesign, TwoWindingModel

path = files("data").joinpath("1250kVA_sc_transformer.json")

import json

with open(path) as json_file:
    data = json.load(json_file)

transformer = TransformerDesign.from_dict(data)

trafo_model = TwoWindingModel(input=transformer)
trafo_model.calculate(is_sc=True)
print(trafo_model.lv_winding)

WindingDesign(inner_radius=205.0, thickness=13.41541620852261, winding_height=342.5, filling_factor=100.0, current_density=4.1, mass=117.34983804213182, dc_loss=0.0, ac_loss=18947591686442.832, outer_radius=218.4154162085226, cable_length=14426366793141.701)


In [5]:
# FEM calculation
trafo_model.fem_simulation()

Mesh Generator: Initial mesh generation
Mesh generator: Triangle
Mesh Generator: Reading initial mesh from memory
Solver: Solver - MUMPS (none)
Solver: Adaptivity step: 1 (error: 100 %, DOFs: 80841)
Solver: Solver - MUMPS (none)
Solver: Adaptivity step: 2 (error: 484.577 %, DOFs: 71395)
Solver: Solver - MUMPS (none)
Solver: Adaptivity step: 3 (error: 1.37177e-06 %, DOFs: 79731)
Solver: Elapsed time: 00:05.809 s
47.5176863269466
L: 2.9194866479275986e-05
zb, ib: 0.12800000000000003 1804.2195912175807
7.165498285455338
0.06368600018945325 0.046823955996588396


The following command can list the main results of the calculation:

In [6]:
print(trafo_model.results)
print('analytical sci:', trafo_model.results.sci)

MainResults(wh=809.5, feasible=True, core_loss=2.063667892391339, load_loss=615055003917038.9, copper_mass=0.0, sci=0.05264094474342352, window_width=140.97862558312448, core_mass=2794.3887336839307, turn_voltage=22.11771077677107, capitalized_cost=0.0, fem_based_sci=7.165498285455338, fem_bax=0.06368600018945325, fem_brad=0.046823955996588396)
analytical sci: 0.05264094474342352


The resulting fem-based short circuit impedance is 2.28 %, which is very close to the given 2.45 %, however, the analytically calculated sci is only 1.7%. The reason is that the analytical formula contains the turn voltage, which is different, the core somehow not sized properly. 

In [7]:
print(trafo_model.hv_winding)

WindingDesign(inner_radius=252.91541620852266, thickness=8.563209374601865, winding_height=354.83, filling_factor=100.0, current_density=6.2, mass=94.27660289270821, dc_loss=0.0, ac_loss=15222130741711.072, outer_radius=261.47862558312454, cable_length=11589865619186.201)


In [8]:
print(trafo_model.lv_winding)

WindingDesign(inner_radius=205.0, thickness=13.41541620852261, winding_height=342.5, filling_factor=100.0, current_density=4.1, mass=117.34983804213182, dc_loss=0.0, ac_loss=18947591686442.832, outer_radius=218.4154162085226, cable_length=14426366793141.701)


In [9]:
i1 = 909.32
i2 = 34.64
u1 = 0.4
u2 = 10.5
s1 = 1.73 * u1 * i1
ratio = i1/i2
print(s1)
print(10500/262/1.73)
print(400/10/1.73)

629.24944
23.16551206812867
23.121387283236995


In [10]:
jlv=i1*10/8/342.5
print(jlv)

3.3186861313868614


In [11]:
print(i2*262/13.5/355)


1.8937256129368805
